<a href="https://colab.research.google.com/github/Guiils/analise_de_dados/blob/main/projeto_Alesp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Declaração de libs

In [ ]:
pip install gcsfs

In [ ]:
pip install pandera

In [ ]:
import pandas as pd
import os
from google.cloud import storage
import numpy as np
import pandera as pa

## Configurando a parte visual do DataFrame pelo Pandas

In [ ]:
pd.set_option('display.max_columns', 100)

## Conector para Cloud Storage(bucket)

In [ ]:
serviceAccount = '/content/advance-airline-377819-1bf613aac07c.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
client = storage.Client()
bucket = client.get_bucket('ad1-bucket')
bucket.blob('depsp.csv')
path = 'gs://ad1-bucket/brutos/depsp.csv'

In [ ]:
path = 'gs://ad1-bucket/brutos/depsp.csv'

## Extração de Dados(Extract)

In [ ]:
df = pd.read_csv(path)
dfback = df.copy()

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Pré análise

In [ ]:
df.head(5)

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA


In [ ]:
df.dtypes

Ano             int64
Matricula       int64
Mes             int64
Valor         float64
CNPJ           object
Deputado       object
Tipo           object
Fornecedor     object
dtype: object

In [ ]:
pd.unique(df['cnpj'])

In [ ]:
df.isna().sum()

## Transformações(Limpeza,conversão,etc)

In [ ]:
df.rename(columns={'Ano':'ano','Matricula':'matricula','Mes':'mes','Valor':'valor','CNPJ':'cnpj','Deputado':'deputado','Tipo':'tipo','Fornecedor':'fornecedor'}, inplace=True)

## Criação de Filtros

In [ ]:
filtro4anos = df.ano > 2017
anos4 = df.loc[filtro4anos]
primeiroSem = df.mes <= 6
segundoSem = df.mes >= 7
gastosPrimeiroSem = df.loc[primeiroSem & filtro4anos]
gastosSegundoSem = df.loc[segundoSem & filtro4anos]

In [ ]:
anos4

In [ ]:
anos4.isna().sum()

ano              0
matricula        0
mes              0
valor            0
cnpj          1483
deputado         0
tipo             0
fornecedor       0
dtype: int64

##Insights

In [ ]:
#Os que mais gastaram no tempo total do df
gastoTotal = df.groupby('deputado')['valor'].sum().sort_values(ascending=False)

In [ ]:
#Os que mais gastaram
gastoTotal.head(3)

deputado
CAMPOS MACHADO       5691197.91
ENIO TATTO           5363451.90
MARIA LÚCIA AMARY    5297913.91
Name: valor, dtype: float64

In [ ]:
#Os que menos gastaram
gastoTotal.tail(3)

deputado
ULISSES SALES                   2653.53
MAJOR MAURO                     2311.53
ANTÔNIO AGUIMARÃES DE CALDAS    2225.63
Name: valor, dtype: float64

In [ ]:
#Os que mais gastaram nos ultimos 4 anos
gasto4Anos = anos4.groupby('deputado')['valor'].sum().sort_values(ascending=False)

In [ ]:
#Os que mais gastaram
gasto4Anos.head(3)

deputado
TEONILIO BARBA    1917275.47
MARTA COSTA       1865500.77
ENIO TATTO        1860088.08
Name: valor, dtype: float64

In [ ]:
#Os que menos gastaram
gasto4Anos.tail(3)

deputado
EVANDRO LOSACCO        2818.42
MAJOR MAURO            2311.53
LUIZ GONZAGA VIEIRA    2251.96
Name: valor, dtype: float64

In [ ]:
#Ranking de maiores gastos por tipo
anos4.groupby('tipo')['valor'].sum().sort_values(ascending=False)

tipo
L - LOCAÇÃO DE BENS IMÓVEIS                                           20261433.46
D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS    16305196.99
O - LOCAÇÃO DE VEÍCULO                                                11693211.97
F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)      10818575.26
K - LOCAÇÃO DE BENS MÓVEIS                                             9211102.83
A - COMBUSTÍVEIS E LUBRIFICANTES                                       8377506.79
M - MANUTENÇÃO DE BENS MÓVEIS,  IMÓVEIS, CONDOMÍNIOS E OUTROS          6115481.49
H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)       5566687.31
E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO              5213792.32
J - SERVIÇOS DE COMUNICAÇÃO                                            3766041.38
I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO                    3708313.49
N - MORADIA                                                            2898985.17
G - ASSINAT

In [ ]:
gastosPrimeiroSem.groupby('tipo')['valor'].sum().sort_values(ascending=False)

tipo
L - LOCAÇÃO DE BENS IMÓVEIS                                           10289446.28
D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS     8781729.01
O - LOCAÇÃO DE VEÍCULO                                                 5613922.56
F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)       5461478.93
K - LOCAÇÃO DE BENS MÓVEIS                                             4574769.73
A - COMBUSTÍVEIS E LUBRIFICANTES                                       4233214.66
M - MANUTENÇÃO DE BENS MÓVEIS,  IMÓVEIS, CONDOMÍNIOS E OUTROS          3237950.18
H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)       2946087.97
E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO              2573949.45
J - SERVIÇOS DE COMUNICAÇÃO                                            1974243.19
I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO                    1780333.01
N - MORADIA                                                            1480775.38
G - ASSINAT

In [ ]:
gastosSegundoSem.groupby('tipo')['valor'].sum().sort_values(ascending=False)

tipo
L - LOCAÇÃO DE BENS IMÓVEIS                                           9971987.18
D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS    7523467.98
O - LOCAÇÃO DE VEÍCULO                                                6079289.41
F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)      5357096.33
K - LOCAÇÃO DE BENS MÓVEIS                                            4636333.10
A - COMBUSTÍVEIS E LUBRIFICANTES                                      4144292.13
M - MANUTENÇÃO DE BENS MÓVEIS,  IMÓVEIS, CONDOMÍNIOS E OUTROS         2877531.31
E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO             2639842.87
H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)      2620599.34
I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO                   1927980.48
J - SERVIÇOS DE COMUNICAÇÃO                                           1791798.19
N - MORADIA                                                           1418209.79
G - ASSINATURAS DE PERI

In [ ]:
relatorioGastosPrimSem = gastosPrimeiroSem.groupby('deputado')['valor'].sum().sort_values(ascending=False)

In [ ]:
relatorioGastosSegSem = gastosSegundoSem.groupby('deputado')['valor'].sum().sort_values(ascending=False)

##LOAD(Carregamento) salvar o df.

In [ ]:
relatorioGastosPrimSem.to_csv('gs://ad1-bucket/tratados/relatoriogastos1sem.csv', index=False)